In [1]:
from sqlalchemy import create_engine
from sqlalchemy.engine import URL
from urllib.parse import unquote

server_name   = "localhost"
database_name = "Bokhandel"

connection_string = f"DRIVER=ODBC Driver 17 for SQL Server;SERVER={server_name};DATABASE={database_name};Trusted_Connection=yes"
url_string        = URL.create("mssql+pyodbc", query={"odbc_connect": connection_string})

print('Connecting to database using URL string:')
unquoted_url = unquote(str(url_string))
print(unquoted_url, '\n')

try:    
    engine = create_engine(url_string)
    with engine.connect() as connection:
        print(f'Successfully connected to {database_name}!')
except Exception as e:
    print('Error while connecting to database:\n')
    print(e)

Connecting to database using URL string:
mssql+pyodbc://?odbc_connect=DRIVER=ODBC+Driver+17+for+SQL+Server;SERVER=localhost;DATABASE=Bokhandel;Trusted_Connection=yes 

Successfully connected to Bokhandel!


In [34]:
from sqlalchemy import text

def search_books(search_term, connection):
    query = text("""
    SELECT b.Titel, ls.ButikID, bu.Butiksnamn, ls.Antal
    FROM Böcker b
    JOIN LagerSaldo ls ON b.ISBN13 = ls.ISBN
    JOIN Butiker bu ON ls.ButikID = bu.ButikID
    WHERE b.Titel LIKE :search_term
    """)
    results = connection.execute(query, {'search_term': f'%{search_term}%'}).fetchall()

    return results

search_term = input("Ange Boktitel: ").strip()

if search_term:
    with engine.connect() as connection:
        results = search_books(search_term, connection)
        for result in results:
            print(f"Boktitel: {result.Titel}, ButikID: {result.ButikID}, Butiknamn: {result.Butiksnamn}, Antal exemplar: {result.Antal}")
else: # doesn't display entire table if no search term is given
    print("Ingen sökterm angiven. Avslutar programmet.")

Ingen sökterm angiven. Avslutar programmet.
